Tujuan Tugas Besar : 
- Melakukan Clustering terhadap data yang ada untuk menentukan kelompok / cluster data.

- Melakukan Regresi (prediksi) terhada O3 di suatu daerah.

In [1]:
from flask import Flask, request, render_template, url_for, redirect, jsonify
import tensorflow as tf
import pandas as pd
from math import sqrt
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import r2_score
from sklearn import preprocessing

In [2]:
def neural_network(filename):
    x = pd.read_csv(filename)
    y = x.pop('O3')
    
    x = preprocessing.scale(x)
    model = tf.keras.models.load_model('ozone-final.model')
    pred = model.predict(x)

    mse_train = mean_squared_error(y, pred)
    rmse_train = sqrt(mean_squared_error(y, pred))
    mae_train = mean_absolute_error(y, pred)
    r2 = "{:0.3f}".format(r2_score(y, pred))
    
    return mse_train, rmse_train, mae_train, r2

In [3]:
def cluster(filename, param, clus):
    clus = int(clus)
    x = pd.read_csv(filename)
    x = x.drop(['date', 'Address'], axis=1)

    X_after, test = train_test_split(x, test_size=0.2, random_state=42)

    colors = 10*[
        'green', 'red', 'cyan', 'blue', 'black', 'yellow', 'magenta'
    ]
    x2 = X_after[['Latitude','Longitude', param]].copy()
    x_array = np.array(x2)

    wcss = []

    for i in range(1,11):
        kmeans = KMeans(i)
        kmeans.fit(x_array)
        wcss_iteration = kmeans.inertia_
        wcss.append(wcss_iteration)

    number_of_clusters = range(1,11)
    plt.plot(number_of_clusters,wcss)
    plt.xlabel('NUMBER OF CLUSTERS')
    plt.ylabel('WCSS value')
    plt.show

    clf = K_Means()
    clf.fit(x_array, clus)

    for classification in clf.classifications:
        color = colors[classification]
        for featureset in clf.classifications[classification]:
            list_class.append(classification)
    x['Cluster'] = list_class
    x.to_csv(r'data_hasil_cluster.csv', index = False, header=True)

    return list_class

In [4]:
class K_Means:
    def __init__(self, tol=0.001, max_iter=300):
        self.tol = tol
        self.max_iter = max_iter

    def fit(self, data, k):

        self.centroids = {}

        for i in range(k):
            self.centroids[i] = data[i]

        for i in range(self.max_iter):

            self.classifications = {}

            for i in range(k):
                self.classifications[i] = []

            for featureset in data:
                distances = [
                    np.linalg.norm(
                        featureset-self.centroids[centroid]
                    ) 
                        for centroid in self.centroids
                    ]
                classification = distances.index(min(distances))
                self.classifications[classification].append(featureset)

            prev_centroids = dict(self.centroids)

            for classification in self.classifications:
                self.centroids[classification] = np.average(
                    self.classifications[classification], axis=0
                )
            optimized = True

            for c in self.centroids:
                original_centroid = prev_centroids[c]
                current_centroid = self.centroids[c]
                if np.sum((current_centroid-original_centroid)/original_centroid*100.0) > self.tol:
                    optimized = False

            if optimized:
                break

    def predict(self, data):
        distances = [
            np.linalg.norm(
                data-self.centroids[centroid]
            ) for centroid in self.centroids
        ]

        classification = distances.index(min(distances))

        return classification

In [5]:
ui = Flask(__name__)

x = []
@ui.route('/', methods=['POST', 'GET'])
def index():
    if request.method == 'POST':
        filename = request.form['myfile']
        mse, rmse, mae, r2 = neural_network(filename) 
        df = pd.read_csv(filename)
        return render_template('home.html', mse_dis = mse, rmse_dis = rmse, mae_dis = mae, r2_dis = r2, tables=[df.to_html(classes='data')], titles=df.columns.values)
    else:
        return render_template('home.html')

@ui.route('/cluster', methods=['POST', 'GET'])
def cluster():
    if request.method == 'POST':
        filename = request.form['myfile']
        param = request.form.get('colours')
        clus = request.form['k']
        classification = cluster(filename, param, clus)
        df = pd.read_csv('data_hasil_cluster.csv')
        return render_template('cluster.html', tables=[x.to_html(classes='data')], titles=x.columns.values)
    else:
        x = pd.read_csv('data_cleaned_avg_cluster2.csv')
        x = x.drop(['date', 'Address','Longitude', 'Latitude'], axis=1)
        colours = []
        for col in x.columns: 
             colours.append(col)
        return render_template('cluster.html', colours=colours)
        
if __name__ == "__main__":
    ui.run()

* Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off
I0430 07:14:04.510501 13336 _internal.py:122]  * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
I0430 07:14:13.340691  7980 _internal.py:122] 127.0.0.1 - - [30/Apr/2020 07:14:13] "GET /cluster HTTP/1.1" 200 -
E0430 07:14:27.718511  7068 app.py:1892] Exception on /cluster [POST]
Traceback (most recent call last):
  File "C:\Users\Asus\AppData\Local\Programs\Python\Python37\lib\site-packages\flask\app.py", line 2447, in wsgi_app
    response = self.full_dispatch_request()
  File "C:\Users\Asus\AppData\Local\Programs\Python\Python37\lib\site-packages\flask\app.py", line 1952, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "C:\Users\Asus\AppData\Local\Programs\Python\Python37\lib\site-packages\flask\app.py", line 1821, in handle_user_exception
    reraise(exc_type, exc_value, tb)
  File "C:\Users\Asus\AppData\Local\Programs\P